In [2]:
import torch
import wandb
from datetime import datetime
import shutil
import importlib
import os
from importlib.machinery import SourceFileLoader
import pandas as pd
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, Dataset, RandomSampler, SequentialSampler
import random
from torch.nn.utils.rnn import pad_sequence
from torch import nn

In [3]:
from transformers import (
    MODEL_WITH_LM_HEAD_MAPPING,
    WEIGHTS_NAME,
    AdamW,
    AutoConfig,
    AutoModelWithLMHead,
    AutoTokenizer,
    PreTrainedModel,
    PreTrainedTokenizer,
    get_linear_schedule_with_warmup,
)

In [4]:
SPECIAL_TOKENS = [
	'<speaker_1>', 
	'</speaker_1>', 
	
	'<speaker_2>',
	'</speaker_2>',
	
	'<persona>',
	'</persona>'
]

SPECIAL_TOKENS = {item:item for item in SPECIAL_TOKENS}

In [5]:
class BaseExperiment:
    def __init__(self, 
        model=None, 
        tokenizer=None, 
        dataloader_train=None,
        dataloader_valid=None,
        dataloader_test=None,
        loss_func_class=None,
        estimate_func_class=None,
        experiment_config=None,
        optimizer_class=None,
        sheduler_class=None,
        project_name=None,
        notebook_name=None,
        name_run="",
        model_description="",
        do_unit_tests=True
        ): 
        assert notebook_name != None, f"notebook_name should be valid filename, but get {notebook_name}"

        # datasets
        self.dataloader_train = dataloader_train
        self.dataloader_valid = dataloader_valid
        self.dataloader_test = dataloader_test
        
        # wandb
        self.notebook_name = notebook_name
        self.project_name = project_name 
        self.experiment_config = experiment_config
        self.wandb_run = None
        self.name_run = name_run
        self.model_description = model_description
        self.model_name = "pytorch_model"
        self.pure_model_name = "pytorch_model"
        self.model_artifact = None

        self.optimizer_class = optimizer_class
        self.sheduler_class = sheduler_class
        self.loss_func_class = loss_func_class
        self.estimate_func_class = estimate_func_class

        self.model = model
        self.tokenizer = tokenizer
        self.optimizer = None
        self.sheduler = None
        self.loss_func = None
        self.estimate_func = None
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        # self.device = torch.device('cpu')
        print(f"Using device {self.device}")

        # prepare for experiment
        self.setup()
        if do_unit_tests:
            self.unit_tests()

    def setup(self):
        self.model.to(self.device)
        self.model.transformer.resize_token_embeddings(len(self.tokenizer))

        self.optimizer = self.optimizer_class(self.model.parameters(), **self.experiment_config['optimizer'])
        
        if self.sheduler_class != None:
            self.sheduler = self.sheduler_class(self.optimizer, **self.experiment_config['sheduler'])

        self.loss_func = self.loss_func_class()
        self.estimate_func = self.estimate_func_class()

        # set model name
        date_time = self.get_date()
        self.model_name = f"{self.name_run}---{date_time}.pt"
        self.pure_model_name = f"{self.name_run}---{date_time}"

        self.experiment_config['model_name'] = self.model_name
        self.experiment_config['pure_model_name'] = self.pure_model_name

        # setup wandb
        # save model structure and weights to wandb
        self.model_artifact = wandb.Artifact(
            self.name_run, type="model",
            description=self.model_description,
            metadata=self.experiment_config)


    def get_date(self):
        now = datetime.now()
        date_time = now.strftime("%m_%d_%Y__%H:%M:%S")
        return date_time

    def unit_tests(self):
        # test training
        X, y = next(iter(self.dataloader_train))
        X, y = X.to(self.device), y.to(self.device)

        pred = self.model(X)
        pred = pred[..., :-1, :].contiguous().view(-1, pred.size(-1))
        y = y[..., 1:].contiguous().view(-1)

        loss = self.loss_func(pred, y)
        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()

        # test valid
        X, y = next(iter(self.dataloader_valid))
        X, y = X.to(self.device), y.to(self.device)
        pred = self.model(X)
        pred = pred[..., :-1, :].contiguous().view(-1, pred.size(-1))
        y = y[..., 1:].contiguous().view(-1)
        test_loss = self.estimate_func(pred, y).item()
        # correct = (pred.argmax(1) == y).type(torch.float).sum().item()

        # initial validation
        self.model.eval()
        test_loss, correct = 0, 0
        num_batches = len(self.dataloader_valid)
        size = len(self.dataloader_valid.dataset)

        with torch.no_grad():
            for X, y in self.dataloader_valid:
                X, y = X.to(self.device), y.to(self.device)
                pred = self.model(X)
                pred = pred[..., :-1, :].contiguous().view(-1, pred.size(-1))
                y = y[..., 1:].contiguous().view(-1)
                test_loss += self.estimate_func(pred, y).item()
                # correct += (pred.argmax(1) == y).type(torch.float).sum().item()

        test_loss /= num_batches
        # correct /= size
        print("Initial val = ", test_loss)

        print("tests ok")


    def train(self):
        # https://colab.research.google.com/github/wandb/examples/blob/master/colabs/wandb-artifacts/Pipeline_Versioning_with_W%26B_Artifacts.ipynb#scrollTo=qrAWbBV1rd4I
        # если попытаться создать переменную чтобы не городить тут код возникает ошибка с wandb!
        with wandb.init(project=self.project_name, entity="dimweb",
                        settings=wandb.Settings(
                            start_method="thread", 
                            # symlink=False
                            ),
                        reinit=True,
                        name=self.name_run,
                        config=self.experiment_config,
                        # sync_tensorboard=True
                        ) as run:

            self.run = run
            
            # save model class
            self.save_model_class()

            # start train
            epochs = self.experiment_config['epochs']
            for i in range(epochs):
                print(f"Epoch: {i}")
                self.train_steps()
                self.valid_steps()
            
            # sync model
            self.wandb_save_model()
            
            print(f"train end")
    
    def save_model_class(self):
        # save class
        model_class_name = self.experiment_config['model_class_name']
        class_script_path_dest = f"{os.path.join(wandb.run.dir, model_class_name)}.py"
        class_script_path_src = f"./models/{model_class_name}.py"
        shutil.copy2(class_script_path_src, class_script_path_dest)
        self.model_artifact.add_file(class_script_path_dest)
        wandb.save(class_script_path_dest)

    def wandb_save_model(self):
        # wandb использует symlinks для того чтобы сохранять файлы
        # но из-за проблем с правами доступа возникает ошибка и модель нельзя сохранить
        # поэтому пришлось сохранять модель в дирректорию с самим запуском
        # https://docs.wandb.ai/guides/track/advanced/save-restore#example-of-saving-a-file-to-the-wandb-run-directory
        # model_save_path = os.path.join(wandb.run.dir, self.model_name)
        # optimizer_save_path = os.path.join(wandb.run.dir, f"optimizer.pt")
        # sheduler_save_path = os.path.join(wandb.run.dir, f"scheduler.pt")

        self.model.save_pretrained(wandb.run.dir)
        self.tokenizer.save_pretrained(wandb.run.dir)
        
        # torch.save(self.model.state_dict(), model_save_path)
        # torch.save(self.optimizer.state_dict(), optimizer_save_path)
        # torch.save(self.sheduler.state_dict(), sheduler_save_path)

        # self.model_artifact.add_file(model_save_path)
        # self.model_artifact.add_file(optimizer_save_path)
        # self.model_artifact.add_file(sheduler_save_path)

        # wandb.save(model_save_path)
        # wandb.save(optimizer_save_path)
        # wandb.save(sheduler_save_path)

        self.experiment_config['saved_path'] = wandb.run.dir

        # save notebook
        notebook_path = os.path.join(wandb.run.dir, self.notebook_name)
        shutil.copy2(self.notebook_name, notebook_path)
        self.model_artifact.add_file(notebook_path)
        wandb.save(notebook_path)

        wandb.log_artifact(self.model_artifact)
    
    def train_steps(self):
        raise NotImplementedError("You need specify training steps")

    def valid_steps(self):
        raise NotImplementedError("You need specify valid steps")

    def load_model(self, artifact_name, additional_model_args={}):
        assert artifact_name != ""
        with wandb.init(project=self.project_name, job_type="inference"):
            model_artifact = wandb.use_artifact(artifact_name)
            model_dir = model_artifact.download()
            model_config = model_artifact.metadata
            model_path = os.path.join(model_dir, model_config['model_name'])
            # print(model_config)

            model_class_name = model_config['model_class_name']
            model_script_path = f"./artifacts/{artifact_name}/{model_class_name}.py"
            # get module by path https://stackoverflow.com/questions/67631/how-to-import-a-module-given-the-full-path?rq=1 
            model_class = getattr(SourceFileLoader(model_class_name, model_script_path).load_module(), model_class_name)
            
            model_args = model_config['model_args']
            model = model_class(**model_args, **additional_model_args)

            model.load_state_dict(torch.load(model_path))
            self.model = model
            self.model.to(self.device)
    
    @staticmethod
    def static_load_model(artifact_name="", project_name="", additional_model_args={}):
        assert artifact_name != ""
        assert project_name != ""
        with wandb.init(project=project_name, job_type="inference"):
            model_artifact = wandb.use_artifact(artifact_name)
            model_dir = model_artifact.download()
            model_config = model_artifact.metadata
            model_path = os.path.join(model_dir, model_config['model_name'])

            model_class_name = model_config['model_class_name']
            model_script_path = f"./artifacts/{artifact_name}/{model_class_name}.py" 
            model_class = getattr(SourceFileLoader(model_class_name, model_script_path).load_module(), model_class_name)
            
            model_args = model_config['model_args']
            model = model_class(**model_args, **additional_model_args)
            device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
            model.to(device)
            model.load_state_dict(torch.load(model_path))
            
            return model

    def test(self, artifact_name="", model=None):
        raise NotImplementedError("You need specify test steps")


class Experiment(BaseExperiment):
    def __init__(self, **kwargs): 
        super(Experiment, self).__init__(**kwargs)
    
    def train_steps(self):
        self.model.train()
        interval = self.experiment_config['check_interval']

        for batch, (X, y) in enumerate(self.dataloader_train):
            # Send data to training device
            X, y = X.to(self.device), y.to(self.device)
            
            # Compute prediction error
            pred = self.model(X)
            pred = pred[..., :-1, :].contiguous().view(-1, pred.size(-1))
            y = y[..., 1:].contiguous().view(-1)
            loss = self.loss_func(pred, y)
            
            # Backpropagation
            self.optimizer.zero_grad()
            loss.backward()
            self.optimizer.step()
            
            if self.sheduler != None:
                self.sheduler.step()
            
            # Progress output
            if batch % interval == 0:
                wandb.log({"train_loss": loss.item()})

    def valid_steps(self):
        self.model.eval()
        test_loss, correct = 0, 0
        num_batches = len(self.dataloader_valid)
        size = len(self.dataloader_valid.dataset)

        with torch.no_grad():
            for X, y in self.dataloader_valid:
                X, y = X.to(self.device), y.to(self.device)
                pred = self.model(X)
                pred = pred[..., :-1, :].contiguous().view(-1, pred.size(-1))
                y = y[..., 1:].contiguous().view(-1)
                test_loss += self.estimate_func(pred, y).item()
                # correct += (pred.argmax(1) == y).type(torch.float).sum().item()

        test_loss /= num_batches
        # correct /= size
        
        wandb.log({"val_loss": test_loss})
        # wandb.log({"val_acc": correct})

    def test(self, artifact_name="", persona="", user_inputs=None):
        with wandb.init(project="gpt_persona_bot", job_type="inference"):
            model_artifact = wandb.use_artifact(artifact_name)
            model_dir = model_artifact.download()
            model_config = model_artifact.metadata
            model_folder = model_config['saved_path'] 
            print(model_config)

            model_class_name = model_config['model_class_name']
            model_script_path = f"./artifacts/{artifact_name}/{model_class_name}.py"
            # get module by path https://stackoverflow.com/questions/67631/how-to-import-a-module-given-the-full-path?rq=1 
            model_class = getattr(SourceFileLoader(model_class_name, model_script_path).load_module(), model_class_name)
            
            self.model = model_class.from_pretrained(model_folder)
            self.tokenizer = AutoTokenizer.from_pretrained(model_folder)
            print("Start conversation")
            print(persona)
            persona = f"{SPECIAL_TOKENS['<persona>']} {persona} {SPECIAL_TOKENS['</persona>']}"
            chat_history_ids = self.tokenizer.encode(persona, return_tensors='pt')
            VOCAB_TOKENS = self.tokenizer.get_added_vocab()

            last_index = 0

            for step in range(len(user_inputs)):
                print(f"User: {user_inputs[step]}")

                user_input = f"{SPECIAL_TOKENS['<speaker_1>']} {user_inputs[step]} {SPECIAL_TOKENS['</speaker_1>']}{SPECIAL_TOKENS['<speaker_2>']}"
                new_user_input_ids = self.tokenizer.encode(user_input, return_tensors='pt')
                bot_input_ids = torch.cat([chat_history_ids, new_user_input_ids], dim=-1)

                # generated a response while limiting the total chat history to 1000 tokens, 
                model_response = self.model.generate(
                    bot_input_ids, max_length=250,
                    pad_token_id=self.tokenizer.eos_token_id,  
                    no_repeat_ngram_size=3,       
                    do_sample=True, 
                    top_k=100,
                    top_p=0.7,
                    temperature = 0.8,
                )

                model_response_list = list(model_response[0])
                end_speaker_index = last_index+model_response_list[last_index+1:].index(VOCAB_TOKENS["</speaker_2>"]) +1
                last_index = end_speaker_index
                model_response = model_response[:, :end_speaker_index+1]

                chat_history_ids = model_response
                print(f"Bot: {self.tokenizer.decode(chat_history_ids[0][len(bot_input_ids[0])-1:], skip_special_tokens=True)}")

                print("test end")

## Создаем датасет

In [6]:
persona_chat_original = pd.read_csv("./persona_chat.csv")
persona_chat_original = persona_chat_original[:16]
persona_chat_original.head(3)

,Unnamed: 0,Persona,chat
0,0,i like to remodel homes. i like to go hunting...,"hi , how are you doing ? i am getting ready to..."
1,1,my mom is my best friend. i have four sisters...,"hi , how are you doing today ?\ni am spending ..."
2,2,i had a gig at local theater last night. i wo...,"we all live in a yellow submarine , a yellow s..."


In [7]:
class PersonaChatGenerator:
	def __init__(self, 
		initial_dataset=None,
	):
		self.initial_dataset = initial_dataset
		self.processed_dataset = []
		self.process_dataset()

	def process_dataset(self):
		processed_dataset = {
			"persona": [],
			"history": [],
			# "target": []
		}

		speaker_1_start = SPECIAL_TOKENS['<speaker_1>']
		speaker_1_end = SPECIAL_TOKENS['</speaker_1>']
		
		speaker_2_start = SPECIAL_TOKENS['<speaker_2>']
		speaker_2_end = SPECIAL_TOKENS['</speaker_2>']

		for i in range(len(self.initial_dataset)):
			persona = self.initial_dataset['Persona'].iloc[i]
			persona = f"{SPECIAL_TOKENS['<persona>']} {persona} {SPECIAL_TOKENS['</persona>']}"
			chat = self.initial_dataset['chat'].iloc[i].split("\n")
			chat = chat[:-1]
			history = ""
			for j in range(len(chat)):
				reply = chat[j]
				if (j+1) % 2 == 0:
					reply = f"{speaker_2_start} {reply} {speaker_2_end}"
					history += reply

					processed_dataset['persona'].append(persona)
					processed_dataset['history'].append(history)
					# processed_dataset['target'].append(reply)

				else:
					reply = f"{speaker_1_start} {reply} {speaker_1_end}"
					history += reply 

		dataset = pd.DataFrame(data=processed_dataset)
		return dataset

train_dataset_csv, valid_dataset_csv = train_test_split(persona_chat_original, test_size=0.1)
train_dataset_csv, valid_dataset_csv = train_dataset_csv.reset_index(), valid_dataset_csv.reset_index()

train_dataset_generator = PersonaChatGenerator(
	initial_dataset=train_dataset_csv,
)

valid_dataset_generator = PersonaChatGenerator(
	initial_dataset=valid_dataset_csv,
)

In [8]:
class PersonaChatDataset(Dataset):
	def __init__(self, 
		initial_dataset=None,
		tokenizer=None
	):
		self.initial_dataset = initial_dataset
		self.tokenizer = tokenizer
	
	def __len__(self):
		return len(self.initial_dataset)
	
	def __getitem__(self, idx):
		row = self.initial_dataset.iloc[idx]
		persona = [item.strip() for item in row['persona'].split(".") if len(item) > 0 ]
		random.shuffle(persona)
		persona = [torch.tensor(self.tokenizer.encode(item)).flatten() for item in persona]
		persona = torch.cat([*persona])

		history = row['history']
		history = self.tokenizer.encode(history)
		history = torch.tensor(history).flatten()
		
		# target = row['target']
		# target = torch.tensor(self.tokenizer.encode(target)).flatten()
		# target = torch.cat([target, torch.tensor([self.tokenizer.eos_token_id], dtype=torch.long)])

		# feature = torch.cat([persona, history, torch.tensor([tokenizer.eos_token_id])])
		feature = torch.cat([persona, history])

		# target = torch.tensor(target, dtype=torch.long)
		# target = target.flatten()
		return {
			"feature": feature,
			"target": feature 
		}


tokenizer = AutoTokenizer.from_pretrained("microsoft/DialoGPT-small")
tokenizer.add_tokens(list(SPECIAL_TOKENS.values()), special_tokens=True)

train_dataset = PersonaChatDataset(
	initial_dataset=train_dataset_generator.process_dataset(),
	tokenizer=tokenizer
)

valid_dataset = PersonaChatDataset(
	initial_dataset=valid_dataset_generator.process_dataset(),
	tokenizer=tokenizer
)

def collate(examples):
	# print("EXAMPLES", examples)
	features = [item['feature'] for item in examples]
	features = pad_sequence(features, batch_first=True)
	
	targets = [item['target'] for item in examples]
	targets = pad_sequence(targets, batch_first=True)

	# return {
	# 	"feature": torch.tensor(features, dtype=torch.long),
	# 	"target": torch.tensor(targets, dtype=torch.long)
	# }
	return torch.tensor(features, dtype=torch.long), torch.tensor(features, dtype=torch.long)

train_dataloader = DataLoader(
    train_dataset, 
	batch_size=4, 
	collate_fn=collate, 
	drop_last = True,
	shuffle=True
)

valid_dataloader = DataLoader(
    valid_dataset, 
	batch_size=8, 
	collate_fn=collate, 
	drop_last=False,
	shuffle=False
)

## Тестируем dumb модель

In [9]:
# google colab не обновляет файлы, поэтому приходится делать это вручную, при помощи такого страшного импорта
def import_class(class_name):
    return getattr(importlib.reload(getattr(__import__(f"models.{class_name}"), class_name)), class_name)

GPT2LMModel = import_class("GPT2LMModel") 
tokenizer = AutoTokenizer.from_pretrained("microsoft/DialoGPT-small")
tokenizer.add_tokens(list(SPECIAL_TOKENS.values()), special_tokens=True)

exp_config = {
    "batch_size": 4,
    "check_interval": 100,
    "epochs": 1,
    "optimizer": {
        "lr": 1e-3
    },
    "model_name": "pytorch_model",
    "model_class_name": str(GPT2LMModel.__name__),
    "model_args": {
        "n_classes": len(tokenizer),
        "base_model_name": "microsoft/DialoGPT-small",
        "emb_size": 768
    },
    "saved_path": ""
}

exp_config["sheduler"] = {
    "max_lr": 0.01, 
    "steps_per_epoch": len(train_dataloader), 
    "epochs": 1
}

model = GPT2LMModel(**exp_config['model_args'])

# не хочу создавать глобальные переменные 
exp_params = {
    "model": model, 
    "tokenizer": tokenizer,
    "dataloader_train": train_dataloader,
    "dataloader_valid": valid_dataloader,
    "dataloader_test": valid_dataloader,
    "loss_func_class": nn.CrossEntropyLoss,
    "estimate_func_class": nn.CrossEntropyLoss,
    "experiment_config": exp_config,
    "optimizer_class": torch.optim.Adam,
    "sheduler_class": None,
    "notebook_name": "gpt_persona_v1.ipynb",
    "project_name": "gpt_persona_bot",
    "name_run": "test_run",
    "model_description": "Test my new model",
    "do_unit_tests": False
}

experiment_test = Experiment(**exp_params)

Some weights of the model checkpoint at microsoft/DialoGPT-small were not used when initializing GPT2Model: ['lm_head.weight']
- This IS expected if you are initializing GPT2Model from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing GPT2Model from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Using device cuda


In [31]:
!ls /home/dimweb/sandbox/persona_bot/gpt_persona_v1/wandb/run-20220708_034230-3rmqo56x/files/

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
added_tokens.json     merges.txt	       tokenizer_config.json
config.json	      output.log	       tokenizer.json
config.yaml	      pytorch_model.bin        vocab.json
GPT2LMModel.py	      requirements.txt	       wandb-metadata.json
gpt_persona_v1.ipynb  special_tokens_map.json  wandb-summary.json


In [38]:
GPT2LMModel.from_pretrained('./files')

OSError: GPT2Config {
  "_name_or_path": "./files",
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_embd": 768,
  "n_head": 12,
  "n_inner": null,
  "n_layer": 12,
  "n_positions": 1024,
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.1,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "torch_dtype": "float32",
  "transformers_version": "4.20.1",
  "use_cache": true,
  "vocab_size": 50257
}
 is not a local folder and is not a valid model identifier listed on 'https://huggingface.co/models'
If this is a private repository, make sure to pass a token having permission to this repo with `use_au

In [ ]:
'/home/dimweb/sandbox/persona_bot/gpt_persona_v1/wandb/run-20220708_034230-3rmqo56x/files'

In [11]:
# experiment_test.train()
# from models import GPT2LMModel
GPT2LMModel = import_class("GPT2LMModel") 
tokenizer = AutoTokenizer.from_pretrained("microsoft/DialoGPT-small")
tokenizer.add_tokens(list(SPECIAL_TOKENS.values()), special_tokens=True)
model = GPT2LMModel(
	base_model_name="microsoft/DialoGPT-small", 
	n_classes=len(tokenizer), 
	emb_size=768
)

model.save_pretrained("./test")
tokenizer.save_pretrained("./test")

model = GPT2LMModel.from_pretrained("./test")
tokenizer = AutoTokenizer.from_pretrained("./test")

Some weights of the model checkpoint at microsoft/DialoGPT-small were not used when initializing GPT2Model: ['lm_head.weight']
- This IS expected if you are initializing GPT2Model from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing GPT2Model from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


OSError: GPT2Config {
  "_name_or_path": "./test",
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_embd": 768,
  "n_head": 12,
  "n_inner": null,
  "n_layer": 12,
  "n_positions": 1024,
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.1,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "torch_dtype": "float32",
  "transformers_version": "4.20.1",
  "use_cache": true,
  "vocab_size": 50257
}
 is not a local folder and is not a valid model identifier listed on 'https://huggingface.co/models'
If this is a private repository, make sure to pass a token having permission to this repo with `use_aut

In [24]:
experiment_test.test(
	artifact_name="test_run:v2"
)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


{'epochs': 1, 'sheduler': {'epochs': 1, 'max_lr': 0.01, 'steps_per_epoch': 25}, 'optimizer': {'lr': 0.001}, 'batch_size': 4, 'model_args': {'emb_size': 768, 'n_classes': 50263, 'base_model_name': 'microsoft/DialoGPT-small'}, 'model_name': 'test_run---07_08_2022__03:42:21.pt', 'saved_path': '/home/dimweb/sandbox/persona_bot/gpt_persona_v1/wandb/run-20220708_034230-3rmqo56x/files', 'check_interval': 100, 'pure_model_name': 'test_run---07_08_2022__03:42:21', 'model_class_name': 'GPT2LMModel'}


OSError: We couldn't connect to 'https://huggingface.co' to load this model, couldn't find it in the cached files and it looks like GPT2Config {
  "_name_or_path": "/home/dimweb/sandbox/persona_bot/gpt_persona_v1/wandb/run-20220708_034230-3rmqo56x/files",
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_embd": 768,
  "n_head": 12,
  "n_inner": null,
  "n_layer": 12,
  "n_positions": 1024,
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.1,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "torch_dtype": "float32",
  "transformers_version": "4.20.1",
  "use_cache": true,
  "vocab_size": 50257
}